# 1. Niveles Lingüísticos

### Cesar Cossio Guerrero

## Tarea 1: Niveles del lenguaje

### FECHA DE ENTREGA: 16 de Febrero 2025 at 11:59pm

### Fonética

1. Si tenemos un sistema de búsqueda que recibe una palabra ortográfica y devuelve sus transcripciones fonológicas, proponga una solución para los casos en que la palabra buscada no se encuentra en el lexicón/diccionario. *¿Cómo devolver o aproximar su transcripción fonológica?*
  - Reutiliza el sistema de búsqueda visto en clase y mejoralo con esta funcionalidad

### Morfología

2. Obtenga los datos de `test` y `dev` para todas las lenguas disponibles en el Shared Task SIGMORPHON 2022 y haga lo siguiente:
    - En un plot de 4 columnas y 2 rows muestre las siguientes distribuciones (un subplot por lengua):
        - Plot 1: distribución de longitud de palabras
        - Plot 2: distribución de la cuenta de morfemas
        - Plot 3: distribución de categorias (si existe para la lengua)
    - Realice una función que imprima por cada lengua lo siguiente:
        - Total de palabras
        - La longitud de palabra promedio
        - La cuenta de morfemas promedio
        - La categoría más común
    - Con base en esta información elabore una conclusión lingüística sobre la morfología de las lenguas analizadas.
    
### EXTRA:

- Imprimir la [matríz de confusión](https://en.wikipedia.org/wiki/Confusion_matrix) para el etiquetador CRFs visto en clase y elaborar una conclusión sobre los resultados

# **Ejercicio 1 Fonética**

## ¿Cómo se elige una representación fonológica parecida a una palabra que no está en el vocabulario? Con la similitud de Jaro-Winkler se encuentra la palabra más parecida en la base de datos y luego se usa la misma función que vimos en clase para encontrar su representación fonológica. La cual es la más parecida a la palabra de entrada. Para ello cree una función que se muestra a continuación

# **Funciones Necesarias para el ejercicio 1**

In [ ]:
!pip install jellyfish
import jellyfish
import http
import requests as r
from pprint import pprint as pp
IPA_URL = "https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/{lang}.txt"

In [ ]:
def download_ipa_corpus(iso_lang: str) -> str:
    """Get ipa-dict file from Github

    Parameters:
    -----------
    iso_lang:
        Language as iso code

    Results:
    --------
    dict:
        Dictionary with words as keys and phonetic representation
        as values for a given lang code
    """
    print(f"Downloading {iso_lang}", end="::")
    response = r.get(IPA_URL.format(lang=iso_lang))
    status_code = response.status_code
    print(f"status={status_code}")
    if status_code != http.HTTPStatus.OK:
        print(f"ERROR on {iso_lang} :(")
        return ""
    return response.text


In [ ]:
def parse_response(response: str) -> dict:
    """Parse text response from ipa-dict to python dict

    Each row have the format:
    [WORD][TAB]/[IPA]/(, /[IPA]/)?

    Parameters
    ----------
    response: str
        ipa-dict raw text

    Returns
    -------
    dict:
        A dictionary with the word as key and the phonetic
        representations as value
    """
    ipa_list = response.rstrip().split("\n")
    result = {}
    for item in ipa_list:
        if item == '':
            continue
        item_list = item.split("\t")
        result[item_list[0]] = item_list[1]
    return result

es_mx_ipa = parse_response(download_ipa_corpus("es_MX"))

In [ ]:
def get_ipa_transcriptions(word: str, dataset: dict) -> list[str]:
    """Search for a word in an IPA phonetics dict

    Given a word this function return the IPA transcriptions

    Parameters:
    -----------
    word: str
        A word to search in the dataset
    dataset: dict
        A dataset for a given language code

    Returns
    -------
    list[str]:
        List with posible transcriptions if any,
        else an empty list
    """
    return dataset.get(word.lower(), "").split(", ")

In [ ]:
def find_most_similar_word(word, dataset):
    """
    Encuentra la palabra más parecida en el dataset a la palabra de entrada usando la similitud de Jaro-Winkler

    Parámetros:
        word: La palabra de entrada para comparar
        dataset: Un diccionario cuyas llaves son las palabras del vocabularios y sus valores son las
                representacions fonéticas.

    Regresa:
        La palabra que más se parece del dataset a la palabra de entrada
        según la similitud de Jaro Winkler.
    """

    max_similarity = 0
    most_similar_word = word
    for dataset_word in dataset:
        similarity = jellyfish.jaro_similarity(word, dataset_word)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_word = dataset_word

    print('La palabra que buscas no está.\nLa palabra más parecida es: \033[1m ',
          most_similar_word,'\033[0m \nY la representación fonológica es: ')
    return most_similar_word


def improved_get_ipa_transcriptions(word, dataset):
    """
    Searches for a word in an IPA phonetics dict.
    If the word is not in the dataset, then the most similar one is used instead using
    Jaro Winkler similarity.

    Args:
        word: The word to search for.
        dataset: The phonetic dictionary.

    Returns:
        A list of IPA transcriptions. If the word is not found, it tries to find the most similar word
        in the dataset and return its transcriptions.
    """
    transcriptions = dataset.get(word.lower(), [])
    if transcriptions:
        return transcriptions.split(", ")

    # Word not found, try to find similar word
    most_similar = find_most_similar_word(word, dataset)
    print('\033[1m ',dataset.get(most_similar.lower(), "").split(", "),'\033[0m')
    return



# **Solución**
### Se hace la prueba sobre la palabra **mayonesa** con una alteración a **mayotesa**. En el primer campo la lista sale vacía. Usnado la función mejorada, el resultado es la representación fonológica de mayonesa


In [ ]:
get_ipa_transcriptions("mayotesa", es_mx_ipa)

In [ ]:
improved_get_ipa_transcriptions("mayotesa", es_mx_ipa)

# **Funciones y librerías necesarias para el ejercicio 2**

In [ ]:
import pandas as pd
LANGS = {
    "ces": "Czech",
    "eng": "English",
    "fra": "French",
    "hun": "Hungarian",
    "spa": "Spanish",
    "ita": "Italian",
    "lat": "Latin",
    "rus": "Russian",
}

# **Ejercicio 2: Morfología**

## A continuación se muestran las gráficas de las distribuciones que se piden. Estas generadas por unas funciones que se cargan a continuación


In [ ]:
def get_track_files(lang: str, track: str = "word") -> list[str]:
    """Genera una lista de nombres de archivo del shared task

    Con base en el idioma y el track obtiene el nombre de los archivos
    para con información reelevante para hacer análisis estadístico.
    Esto es archivos .test y .dev

    Parameters:
    ----------
    lang : str
        Idioma para el cual se generarán los nombres de archivo.
    track : str, optional
        Track del shared task de donde vienen los datos (por defecto es "word").

    Returns:
    -------
    list[str]
        Una lista de nombres de archivo generados para el idioma y la pista especificados.
    """
    return [
        f"{lang}.{track}.test.gold",
        f"{lang}.{track}.dev",
    ]


def get_raw_corpus(files: list) -> list:
    """Descarga y concatena los datos de los archivos tsv desde una URL base.

    Parameters:
    ----------
    files : list
        Lista de nombres de archivos (sin extensión) que se descargarán
        y concatenarán.

    Returns:
    -------
    list
        Una lista que contiene los contenidos descargados y concatenados
        de los archivos tsv.
    """
    result = []
    for file in files:
        print(f"Downloading {file}.tsv", end=" ")
        response = r.get(f"https://raw.githubusercontent.com/sigmorphon/2022SegmentationST/main/data/{file}.tsv")
        print(f"status={response.status_code}")
        lines = response.text.split("\n")
        result.extend(lines[:-1])
    return result

def raw_corpus_to_dataframe(corpus_list: list, lang: str) -> pd.DataFrame:
    """Convierte una lista de datos de corpus en un DataFrame

    Parameters:
    ----------
    corpus_list : list
        Lista de líneas del corpus a convertir en DataFrame.
    lang : str
        Idioma al que pertenecen los datos del corpus.

    Returns:
    -------
    pd.DataFrame
        Un DataFrame de pandas que contiene los datos del corpus procesados.
    """
    data_list = []
    for line in corpus_list:
        try:
            word, tagged_data, category = line.split("\t")
        except ValueError:
            # Caso donde no existe la categoria
            word, tagged_data = line.split("\t")
            category = "NOT_FOUND"
        morphemes = tagged_data.split()
        data_list.append({"words": word, "morph": morphemes, "category": category, "lang": lang})
    df = pd.DataFrame(data_list)
    df["word_len"] = df["words"].apply(lambda x: len(x))
    df["morph_count"] = df["morph"].apply(lambda x: len(x))
    return df

In [ ]:
def Morphology_DataFrames(lang_codes):

    """
    Una función que carga todos la información de los lenguajes
    en un diccionario de dataframes cuya llave es el nombre
    del lenguaje y el valor es el Dataframe correspondiente

    Parámetros:
        lang_codes: Los códigos de los lenguajes
    """

    DF={}
    for i, lang_code in enumerate(lang_codes):
        files = get_track_files(lang_code)
        raw_data = get_raw_corpus(files)
        DF[lang_code] = raw_corpus_to_dataframe(raw_data, lang=lang_code)
    return DF

LF = Morphology_DataFrames(list(LANGS.keys()))

In [ ]:
import matplotlib.pyplot as plt

def plot_morphology_distributions(lang_codes,dfs,Column):

    """
    Una función que grafica para cada lenguaje la distribución
    correspondiente a la Columna Dada

    Parámetros:
        lang_codes: Los códigos de los lenguajes
        dfs: Es un diccionario que contiene dataframes de los morfemas de los lenguajes
        Column: es la característica de la cual se obtendrá la distribución
    """

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    for i, lang_code in enumerate(lang_codes):
        counts = dfs[lang_code][Column].value_counts().head(30)
        axes[i].bar(counts.index, counts.values)
        axes[i].set_xlabel(Column)
        axes[i].set_ylabel('Frequency')
        axes[i].set_title(f'{Column} Frequency Graph for {LANGS[lang_code]}')

    plt.tight_layout()
    plt.show()


In [ ]:
plot_morphology_distributions(list(LANGS.keys()), LF, 'word_len')

In [ ]:
plot_morphology_distributions(list(LANGS.keys()), LF, 'category')

In [ ]:
plot_morphology_distributions(list(LANGS.keys()), LF, 'morph_count')

**A continuación se muestran todas las Estadísticas que se piden en un dataframe**


In [ ]:
def Medidas_Morfologicas(lang, dfs):

    """
    Una función que genera estadísticas de los lenguajes

    Parámetros:
        lang_codes: Los códigos de los lenguajes
        dfs: Es un diccionario que contiene dataframes
        con información acerca de los morfemas de los lenguajes.
    """

    lf = pd.DataFrame(index=list(LANGS.values()),
                      columns=['Total de palabras', 'Longuitud de Palabras Promedio',
                              'Promedio de Cuenta de Morfemas', 'La categoría más común'])

    for lang_code_k,lang_code_v  in lang.items():
        df = dfs[lang_code_k]
        lf.loc[lang_code_v,'Total de palabras'] = len(df["words"].unique())
        lf.loc[lang_code_v,'Longuitud de Palabras Promedio'] = df["word_len"].mean()
        lf.loc[lang_code_v,'Promedio de Cuenta de Morfemas'] = df["morph_count"].mean()
        lf.loc[lang_code_v,'La categoría más común'] = df["category"].mode()[0]

    return lf

Medidas_Morfologicas(LANGS, LF)


## Lo que más me sorprende es que en general  los lenguajes aquí estudiados tienen un comportamiento bastante similar a pesar de pertenecer a ramas lingüísticas muy diferentes. Me soprendió mucho observar que en el fondo comparten muchas similitudes aunque parezcan, se escriban y suenen muy diferentes. Es decir, tienen morfologías y fonologías distintas, pero en el fondo parece que siguieran ciertas reglas.